<div style="  background: linear-gradient(145deg, #0f172a, #1e293b);  border: 4px solid transparent;  border-radius: 14px;  padding: 18px 22px;  margin: 12px 0;  font-size: 26px;  font-weight: 600;  color: #f8fafc;  box-shadow: 0 6px 14px rgba(0,0,0,0.25);  background-clip: padding-box;  position: relative;">  <div style="    position: absolute;    inset: 0;    padding: 4px;    border-radius: 14px;    background: linear-gradient(90deg, #06b6d4, #3b82f6, #8b5cf6);    -webkit-mask:       linear-gradient(#fff 0 0) content-box,       linear-gradient(#fff 0 0);    -webkit-mask-composite: xor;    mask-composite: exclude;    pointer-events: none;  "></div>    <b>CATEGORY EMBEDDINGS & SHARED LAYERS</b>    <br/>  <span style="color:#9ca3af; font-size: 18px; font-weight: 400;">(Advanced Deep Learning with Keras)</span></div>

## Table of Contents
1. [Category Embeddings](#section-1)
2. [Shared Layers](#section-2)
3. [Merge Layers](#section-3)
4. [Fitting and Predicting with Multiple Inputs](#section-4)
5. [Conclusion](#section-5)

---

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 1. CATEGORY EMBEDDINGS</span><br>

### Understanding Embeddings
Category embeddings are a powerful technique in deep learning used to represent categorical data (like team IDs, words, or user IDs) as continuous floating-point vectors.

**Key Characteristics:**
*   **Input:** Integers (representing categories).
*   **Output:** Floats (dense vectors).
*   **Dimensionality:** The embedding layer increases dimensionality. The output is often flattened back to 2D for subsequent layers.

The Embedding layer functions essentially as a **lookup table**. It learns the relationship between categories during the training process.

### Building the Embedding Layer
To implement this in Keras, we use the `Embedding` layer. In this example, we are modeling "Team Strength" based on a team ID.


In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Input, Flatten
from tensorflow.keras.models import Model

# Define the number of unique categories 
num_unique_teams = 10887

# Input Layer: Expects a single integer
input_tensor = Input(shape=(1,), name='Team_ID_Input')

# Embedding Layer: Maps integer -> float
# input_dim needs to match the number of unique categories
embed_layer = Embedding(input_dim=num_unique_teams,
                        output_dim=1,
                        name='Team-Strength-Lookup')

# Connect Input to Embedding
embed_tensor = embed_layer(input_tensor)

# Flatten Layer: Reshapes (Batch, 1, 1) -> (Batch, 1)
flatten_tensor = Flatten(name='Flatten_Output')(embed_tensor)

# Build the Model
model = Model(inputs=input_tensor, outputs=flatten_tensor)

# Display Summary
model.summary()

/Users/faizy/Desktop/applied-deep-learning-keras/.venv/lib/python3.13/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Team_ID_Input (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Team-Strength-Lookup            │ (None, 1, 1)           │        10,887 │
│ (Embedding)                     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Flatten_Output (Flatten)        │ (None, 1)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,887 (42.53 KB)

 Trainable params: 10,887 (42.53 KB)

 Non-trainable params: 0 (0.00 B)


### Detailed Explanation of Arguments
| Argument | Value | Description |
| :--- | :--- | :--- |
| `input_dim` | `num_unique_teams` (10887) | The size of the vocabulary (number of unique categories). |
| `output_dim` | `1` | The size of the vector space for the embedding. |
| `name` | `'Team-Strength-Lookup'` | A custom name for the layer, useful for debugging and visualization. |

<div style="background: #e0f2fe; border-left: 16px solid #0284c7; padding: 14px 18px; border-radius: 8px; font-size: 18px; color: #075985;"> 💡 <b>Tip:</b> The Embedding layer adds a dimension. If you input shape is <code>(batch_size, 1)</code>, the embedding output is <code>(batch_size, 1, output_dim)</code>. The <code>Flatten()</code> layer is required to squeeze this back down to <code>(batch_size, output_dim)</code> for dense layers. </div>


---

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 2. SHARED LAYERS</span><br>

### Concept of Shared Layers
Shared layers allow you to use the **same layer instance** (with the same weights) multiple times within a model. This is a core feature of the Keras Functional API.

**Why use Shared Layers?**
*   **Siamese Networks:** Comparing two inputs using the exact same logic.
*   **Efficiency:** Reduces the number of parameters to learn.
*   **Flexibility:** Allows complex topologies where different inputs undergo the same transformation.

### Implementation
To share a layer, you instantiate the layer object **once**, and then call it on multiple different input tensors.

#### Original Code: Simple Dense Sharing


In [2]:
from tensorflow.keras.layers import Input, Dense

# Define two separate inputs
input_tensor_1 = Input((1,))
input_tensor_2 = Input((1,))

# Define a single layer ONCE
shared_layer = Dense(1)

# Apply the SAME layer to both inputs
output_tensor_1 = shared_layer(input_tensor_1)
output_tensor_2 = shared_layer(input_tensor_2)


### Sharing Multiple Layers (Model Sharing)
You can also treat an entire `Model` as a layer. This allows you to share a complex block of layers (like the Embedding + Flatten block we created in Section 1) across multiple inputs.


In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense
from tensorflow.keras.models import Model

# --- Step 1: Create the Reusable Component (The Embedding Model) ---
n_teams = 100
inp = Input(shape=(1,))
emb = Embedding(input_dim=n_teams, output_dim=1)(inp)
flat = Flatten()(emb)
# This model transforms an integer ID into a strength score
shared_strength_model = Model(inp, flat, name='Shared_Strength_Model')

# --- Step 2: Create the Main Model with Multiple Inputs ---

# Define inputs for two different teams
input_team_1 = Input(shape=(1,), name='Team_1_ID')
input_team_2 = Input(shape=(1,), name='Team_2_ID')

# Apply the shared model to both inputs
# Both outputs now represent the "strength" of the respective teams
strength_1 = shared_strength_model(input_team_1)
strength_2 = shared_strength_model(input_team_2)

print("Shared Layer Output Shapes:")
print(f"Strength 1: {strength_1.shape}")
print(f"Strength 2: {strength_2.shape}")


Shared Layer Output Shapes:
Strength 1: (None, 1)
Strength 2: (None, 1)



---

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 3. MERGE LAYERS</span><br>

### Combining Tensors
Once you have processed multiple inputs (e.g., Team A strength and Team B strength), you often need to combine them to make a final prediction. Keras provides **Merge Layers** for this purpose.

**Common Merge Operations:**

| Layer Type | Function | Use Case |
| :--- | :--- | :--- |
| **Add** | Element-wise addition | ResNet skip connections, combining scores. |
| **Subtract** | Element-wise subtraction | Calculating difference/distance between vectors. |
| **Multiply** | Element-wise multiplication | Attention mechanisms, masking. |
| **Concatenate** | Joins tensors along an axis | Combining different feature sets. |

### The `Add` Layer

#### 2 Inputs

You can merge more than two tensors at once.

In [4]:
from tensorflow.keras.layers import Input, Add


in_tensor_1 = Input((1,))
in_tensor_2 = Input((1,))

# Merge the two tensors via addition
out_tensor = Add()([in_tensor_1, in_tensor_2])

#### 3 Inputs

You can merge more than two tensors at once.

In [5]:
in_tensor_3 = Input((1,))

out_tensor = Add()([in_tensor_1, in_tensor_2, in_tensor_3])

### Creating and Compiling the Merged Model
To finalize the architecture, we wrap the inputs and the merged output into a `Model` and compile it.



This code builds a complete model that learns to perform arithmetic addition.

In [6]:
from tensorflow.keras.layers import Input, Add
from tensorflow.keras.models import Model

# 1. Define Inputs
in_tensor_1 = Input(shape=(1,), name='Input_A')
in_tensor_2 = Input(shape=(1,), name='Input_B')

# 2. Merge Layer (Addition)
# This layer calculates: Input_A + Input_B
out_tensor = Add(name='Add_Layer')([in_tensor_1, in_tensor_2])

# 3. Create Model
# Note: We pass a LIST of inputs
model = Model(inputs=[in_tensor_1, in_tensor_2], outputs=out_tensor)

# 4. Compile
model.compile(optimizer='adam', loss='mean_absolute_error')

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Input_A             │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Input_B             │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Add_Layer (Add)     │ (None, 1)         │          0 │ Input_A[0][0],    │
│                     │                   │            │ Input_B[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


---

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 4. FITTING AND PREDICTING WITH MULTIPLE INPUTS</span><br>

### Training Multi-Input Models
When a model has multiple input layers, you must provide data as a **list of arrays** (or a dictionary mapping input names to arrays).

### Fitting
The syntax for `fit` mirrors the structure of the `Model` definition. If you defined `inputs=[in1, in2]`, you must pass `[data1, data2]` to fit.


In [7]:
model.fit([data_1, data_2], target)

NameError: name 'data_1' is not defined


### Predicting
Similarly, prediction requires a list of Numpy arrays.


In [ ]:
import numpy as np

# Example 1: 1 + 2 = 3
model.predict([np.array([[1]]), np.array([[2]])])

# Example 2: 42 + 119 = 161
model.predict([np.array([[42]]), np.array([[119]])])


### Evaluating
Evaluation follows the same pattern.


In [ ]:
# Evaluating: -1 + -2 = -3
model.evaluate([np.array([[-1]]), np.array([[-2]])], np.array([[-3]]))
# Loss is 0 because the calculation is perfect)


<div style="background: #e0f2fe; border-left: 16px solid #0284c7; padding: 14px 18px; border-radius: 8px; font-size: 18px; color: #075985;"> 💡 <b>Tip:</b> The <code>Add</code> layer has no trainable weights; it simply performs a mathematical operation. Therefore, the model in this specific example doesn't actually need "training" to learn how to add—the architecture <i>is</i> the addition function. However, in most deep learning tasks (like the Team Strength example), you would need to run <code>model.fit()</code> to update the weights in the Embedding or Dense layers. </div>

#### Full Workflow

In [ ]:
import numpy as np
from tensorflow.keras.layers import Input, Add
from tensorflow.keras.models import Model

# --- 1. Build Model ---
input_1 = Input(shape=(1,))
input_2 = Input(shape=(1,))
output = Add()([input_1, input_2])
model = Model([input_1, input_2], output)
model.compile(optimizer='adam', loss='mae')

# --- 2. Predict (No training needed for pure Add layer) ---
print("--- Prediction 1 ---")
# Inputs: 1 and 2
pred_1 = model.predict([np.array([[1]]), np.array([[2]])])
print(f"Input: 1, 2 | Predicted: {pred_1[0][0]}")

print("\n--- Prediction 2 ---")
# Inputs: 42 and 119
pred_2 = model.predict([np.array([[42]]), np.array([[119]])])
print(f"Input: 42, 119 | Predicted: {pred_2[0][0]}")

# --- 3. Evaluate ---
print("\n--- Evaluation ---")
# Check if -1 + -2 equals -3
loss = model.evaluate([np.array([[-1]]), np.array([[-2]])], 
                      np.array([[-3]]), 
                      verbose=1)
print(f"Mean Absolute Error Loss: {loss}")



---

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 5. CONCLUSION</span><br>

### Summary
In this notebook, we explored advanced Keras architecture patterns using the Functional API:

1.  **Category Embeddings:** We learned how to transform integer inputs (categories) into dense floating-point vectors using `Embedding` layers. This is essential for handling categorical variables like Team IDs or words.
2.  **Shared Layers:** We discovered how to reuse a single layer (or an entire sub-model) across multiple inputs. This allows the model to learn a single representation (e.g., "strength") that applies to different entities equally.
3.  **Merge Layers:** We utilized `Add` (and discussed others like `Concatenate`) to combine multiple input streams into a single output tensor.
4.  **Multiple Inputs:** We mastered the syntax for fitting, predicting, and evaluating models that require multiple data arrays (passed as lists).

### Next Steps
*   **Practice:** Try replacing the `Add` layer with a `Subtract` layer to model the *difference* in team strengths.
*   **Application:** Apply this architecture to a real dataset (like sports game results) where you input two Team IDs and try to predict the score difference.
*   **Expansion:** Experiment with `Concatenate` followed by a `Dense` layer to allow the model to learn non-linear interactions between the two inputs, rather than just simple addition.
